In [30]:
import pandas as pd
import datetime as datetime
import os
# ! pip install xlsxwriter --user
from IPython.display import HTML

def df_horizontal(dfs):
    html = '<div style="display:flex">'
    for df in dfs:
        html += '<div style="margin-right: 32px">'
        html += df.to_html()
        html += '</div>'
    html += '</div>'
    display(HTML(html))
    

In [31]:
data_path=r'D:\PythonDev\MyQuantFinProject\Data'
data_file='World-Focus-1Q24.csv'
isPy=False 
isPriceExcel=True

# Specify Period and Timefame to caculate fund performance and

In [32]:
# If you want to see July-Oct, you have to have last trading day of June
from_str='2023-12-01'  
to_str='2024-03-31' 
timeframe='M'

pct_return_col=f"pct_change_{timeframe}"
std_col="std_of_pct_Daily"
# timeframe='W-FRI'

# Input data

In [33]:
if isPy:
    print(f"Data csv file in Path: {data_path}")
    data_file=input("CSV file name : ")

    from_str=input("From_date(2020-10-01) : ") 
    to_str=input("To_date(2022-12-31): ") 
    timeframe=input("Timeframe(W-FRI,M,Q,Y): ") 
  
    print(f"Process data from {from_str} to {to_str} in {timeframe} timeframe")
    press_y=input(f"Press y=True and n=False to run analaystics : ") 
    if press_y.lower()=='y':
     ok=True
    else:
     exit()


In [34]:
report_prefix_name=data_file.split('.')[0]
data_file=os.path.join(data_path,data_file)
if os.path.exists(data_file)==False:
    raise Exception(f"not found {data_file}")
else:
    print(f"Process file {data_file}")
print(data_file)

Process file D:\PythonDev\MyQuantFinProject\Data\World-Focus-1Q24.csv
D:\PythonDev\MyQuantFinProject\Data\World-Focus-1Q24.csv


# Read csv file and extract and transform data as need

In [35]:
df = pd.read_csv(data_file,index_col='Date/Time',parse_dates=['Date/Time'],dayfirst=True)
df.index.set_names('date',inplace=True)
df.sort_index(inplace=True)
df=df.rename(columns={'Ticker':'symbol','close':'price'})
df=df[['symbol','price']]

#test 1
# df=df.query("symbol not in ('BTC-USD')")
# print(df.info())

df.tail(10)

,symbol,price
date,,
2024-03-27,^BSESN,72996.31
2024-03-27,ACWI,110.15
2024-03-27,SPY,523.17
2024-03-28,^STOXX50E,5083.42
2024-03-28,ACWI,110.13
2024-03-28,^N225,40168.07
2024-03-28,^BSESN,73651.35
2024-03-28,SPY,523.07
2024-03-28,VNM,13.90


In [36]:
list_symbol=df['symbol'].unique().tolist() 

print(list_symbol)


['^BSESN', '^STOXX50E', 'SPY', 'VNM', 'ACWI', '^N225']


# Basic

# Resample data and calculate Percentage of change

In [37]:
# colsROC=['symbol','date','pct_change','std_of_daily_pct','return_to_std']
colsROC=['symbol','date',pct_return_col,std_col]
df_Pct_Price=pd.DataFrame(columns=colsROC)
print(df_Pct_Price)

#https://pandas.pydata.org/docs/reference/api/pandas.core.resample.Resampler.aggregate.html
#https://stackoverflow.com/questions/47938372/pandas-dataframe-resample-aggregate-function-use-multiple-columns-with-a-customi
#https://www.alpharithms.com/aggregating-time-series-data-with-pandas-resampling-411212/
# https://sparkbyexamples.com/pandas/pandas-aggregate-functions-with-examples/
for symbol_name in list_symbol:
    print(f"===================================={symbol_name}====================================")
    dfx_symbol=df.query('symbol==@symbol_name')
    dfx_symbol['_pct_compare_prev_']=dfx_symbol['price'].pct_change()
    dfx_symbol=dfx_symbol.loc[from_str:to_str]
    #print(dfx_symbol.tail())
    
    
    dfx_px=dfx_symbol.resample(timeframe).agg({'price': 'last','_pct_compare_prev_':'std'}) 
    dfx_px=dfx_px.rename(columns={'_pct_compare_prev_':std_col})

    dfx_px[pct_return_col]=dfx_px['price'].pct_change()
    dfx_px[pct_return_col]=dfx_px[pct_return_col].round(4)

    dfx_px=dfx_px.dropna() # the first  rown 
    dfx_px=dfx_px.drop(columns=['price']) # remove price becase we are focuse ROC of Price
    
    dfx_px[pct_return_col]=dfx_px[pct_return_col]*100
    dfx_px[std_col]=dfx_px[std_col]*100
    
    # dfx_px[pct_return_col]=dfx_px[pct_return_col]/dfx_px[std_col]
    
    dfx_px=dfx_px.reset_index()
    dfx_px['symbol']=symbol_name
    dfx_px=dfx_px[colsROC]
    
#     print(f"Calculate ROC of {symbol_name}")
    print(dfx_px)
    
    df_Pct_Price=pd.concat([df_Pct_Price,dfx_px],ignore_index=True,axis=0)

C:\Users\pongthsa\AppData\Local\Temp\ipykernel_17004\317653757.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfx_symbol['_pct_compare_prev_']=dfx_symbol['price'].pct_change()
C:\Users\pongthsa\AppData\Local\Temp\ipykernel_17004\317653757.py:18: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  dfx_px=dfx_symbol.resample(timeframe).agg({'price': 'last','_pct_compare_prev_':'std'})
C:\Users\pongthsa\AppData\Local\Temp\ipykernel_17004\317653757.py:39: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude t

Empty DataFrame
Columns: [symbol, date, pct_change_M, std_of_pct_Daily]
Index: []
====================================^BSESN====================================
   symbol       date  pct_change_M  std_of_pct_Daily
0  ^BSESN 2024-01-31         -0.68          0.980966
1  ^BSESN 2024-02-29          1.04          0.569562
2  ^BSESN 2024-03-31          1.59          0.720421
====================================^STOXX50E====================================
      symbol       date  pct_change_M  std_of_pct_Daily
0  ^STOXX50E 2024-01-31          2.80          0.796696
1  ^STOXX50E 2024-02-29          4.93          0.558556
2  ^STOXX50E 2024-03-31          4.22          0.504795
====================================SPY====================================
  symbol       date  pct_change_M  std_of_pct_Daily
0    SPY 2024-01-31          1.59          0.703624
1    SPY 2024-02-29          5.22          0.764592
2    SPY 2024-03-31          2.95          0.638912
====================================V

In [38]:
print("===============List All================================")
print(df_Pct_Price)

===============List All================================
       symbol       date  pct_change_M  std_of_pct_Daily
0      ^BSESN 2024-01-31         -0.68          0.980966
1      ^BSESN 2024-02-29          1.04          0.569562
2      ^BSESN 2024-03-31          1.59          0.720421
3   ^STOXX50E 2024-01-31          2.80          0.796696
4   ^STOXX50E 2024-02-29          4.93          0.558556
5   ^STOXX50E 2024-03-31          4.22          0.504795
6         SPY 2024-01-31          1.59          0.703624
7         SPY 2024-02-29          5.22          0.764592
8         SPY 2024-03-31          2.95          0.638912
9         VNM 2024-01-31         -1.70          1.028443
10        VNM 2024-02-29          6.38          1.128090
11        VNM 2024-03-31          2.89          1.060282
12       ACWI 2024-01-31          0.28          0.666869
13       ACWI 2024-02-29          4.51          0.714834
14       ACWI 2024-03-31          3.26          0.585742
15      ^N225 2024-01-31        

# Total Period : Summarize and Combine return and std

In [39]:
print(f"==================Sum-Return and Avg-Std based on {timeframe}===========================")
print("Sum percentage of fund return")
dfSum_Pct_Price=df_Pct_Price.groupby(['symbol'])[[pct_return_col]].sum()
dfSum_Pct_Price.reset_index(drop=False,inplace=True)
dfSum_Pct_Price[pct_return_col]=dfSum_Pct_Price[pct_return_col].round(1)


print("==============================================================")
print("Average Std of fund ")
dfAvg_Std_Price=df_Pct_Price.groupby(['symbol'])[[std_col]].mean()
dfAvg_Std_Price.reset_index(drop=False,inplace=True)
dfAvg_Std_Price[std_col]=dfAvg_Std_Price[std_col].round(2)

dfValueSummary=pd.merge(left=dfSum_Pct_Price,right=dfAvg_Std_Price,on='symbol')
dfValueSummary.fillna(value=0,inplace=True)
dfValueSummary.set_index('symbol',inplace=True)
dfValueSummary

==================Sum-Return and Avg-Std based on M===========================
Sum percentage of fund return
Average Std of fund 


,pct_change_M,std_of_pct_Daily
symbol,,
ACWI,8.0,0.66
SPY,9.8,0.70
VNM,7.6,1.07
^BSESN,2.0,0.76
^N225,19.4,1.06
^STOXX50E,12.0,0.62


# Timeframe Period 

# Pivot Percent Return & Std and theire Rank To Dataframe

In [40]:
print("==============================================================")
print(f"Pivot Percent Return and Rank {timeframe} To Dataframe")
dfPct=df_Pct_Price.pivot(index='date', columns='symbol', values=pct_return_col)
dfRank=dfPct[dfPct.columns.tolist()].rank(axis=1)
# print(dfPct.info())

Pivot Percent Return and Rank M To Dataframe


In [41]:
print("==============================================================")
print(f"Pivot Daily STD and Rank in {timeframe} and Rank To Dataframe")
dfStd=df_Pct_Price.pivot(index='date', columns='symbol', values=std_col)
dfRankStd=dfStd[dfStd.columns.tolist()].rank(axis=1,ascending=False)


Pivot Daily STD and Rank in M and Rank To Dataframe


In [42]:
df_horizontal([dfRank,dfPct])
df_horizontal([dfRankStd,dfStd])

symbol,ACWI,SPY,VNM,^BSESN,^N225,^STOXX50E
date,,,,,,
2024-01-31,3.0,4.0,1.0,2.0,6.0,5.0
2024-02-29,2.0,4.0,5.0,1.0,6.0,3.0
2024-03-31,5.0,3.0,2.0,1.0,4.0,6.0
symbol,ACWI,SPY,VNM,^BSESN,^N225,^STOXX50E
date,,,,,,
2024-01-31,0.28,1.59,-1.70,-0.68,8.43,2.80
2024-02-29,4.51,5.22,6.38,1.04,7.94,4.93
2024-03-31,3.26,2.95,2.89,1.59,3.07,4.22


symbol,ACWI,SPY,VNM,^BSESN,^N225,^STOXX50E
date,,,,,,
2024-01-31,6.0,5.0,1.0,2.0,3.0,4.0
2024-02-29,4.0,3.0,1.0,5.0,2.0,6.0
2024-03-31,5.0,4.0,2.0,3.0,1.0,6.0
symbol,ACWI,SPY,VNM,^BSESN,^N225,^STOXX50E
date,,,,,,
2024-01-31,0.666869,0.703624,1.028443,0.980966,0.977318,0.796696
2024-02-29,0.714834,0.764592,1.128090,0.569562,1.014105,0.558556
2024-03-31,0.585742,0.638912,1.060282,0.720421,1.173914,0.504795


# Combine Return and Std Rank as Score

In [43]:
print("==============================================================")
print("Combine Both Pct Return and Rank")
print("Return :higher total score , greater performance")
print("Std : higher total score , worse perfromance")
dfRankSummary=pd.concat([
                     dfRank.sum().to_frame(name='sum_return_rank'),
                     dfRankStd.sum().to_frame(name='sum_std_rank'),  
                    ],axis=1)
dfRankSummary.fillna(value=0,inplace=True)
dfRankSummary

Combine Both Pct Return and Rank
Return :higher total score , greater performance
Std : higher total score , worse perfromance


,sum_return_rank,sum_std_rank
symbol,,
ACWI,10.0,15.0
SPY,11.0,12.0
VNM,8.0,4.0
^BSESN,4.0,10.0
^N225,16.0,6.0
^STOXX50E,14.0,16.0


# Sum Rank Value(OK)  # FinalOut Y2024 Update 09April24

In [45]:
dfSumOfRankSummaryY24=dfRankSummary[["sum_return_rank","sum_std_rank"]].copy()
dfSumOfRankSummaryY24['TotalScoreByRankValue']=dfSumOfRankSummaryY24["sum_return_rank"]+dfSumOfRankSummaryY24["sum_std_rank"]
dfSumOfRankSummaryY24=dfSumOfRankSummaryY24.sort_values(by='TotalScoreByRankValue',ascending=False)
dfSumOfRankSummaryY24

,sum_return_rank,sum_std_rank,TotalScoreByRankValue
symbol,,,
^STOXX50E,14.0,16.0,30.0
ACWI,10.0,15.0,25.0
SPY,11.0,12.0,23.0
^N225,16.0,6.0,22.0
^BSESN,4.0,10.0,14.0
VNM,8.0,4.0,12.0


# Convert value score   and rank score to final rank

In [46]:
dfValueSummary['value_return_rank']=dfValueSummary[pct_return_col].rank()
dfValueSummary['value_std_rank']=dfValueSummary[std_col].rank(ascending=False)

In [47]:
# real score
dfRankSummary['return_rank']=dfRankSummary['sum_return_rank'].rank()
dfRankSummary['std_rank']=dfRankSummary['sum_std_rank'].rank()



In [48]:
df_horizontal([dfValueSummary, dfRankSummary])

,pct_change_M,std_of_pct_Daily,value_return_rank,value_std_rank
symbol,,,,
ACWI,8.0,0.66,3.0,5.0
SPY,9.8,0.70,4.0,4.0
VNM,7.6,1.07,2.0,1.0
^BSESN,2.0,0.76,1.0,3.0
^N225,19.4,1.06,6.0,2.0
^STOXX50E,12.0,0.62,5.0,6.0
,sum_return_rank,sum_std_rank,return_rank,std_rank
symbol,,,,
ACWI,10.0,15.0,3.0,5.0


# Summarize Total Score 

In [49]:
dfFinalScore= pd.concat([ dfValueSummary[['value_return_rank','value_std_rank']],dfRankSummary[['return_rank','std_rank']]],axis=1)
dfFinalScore['TotalScore']=dfFinalScore.sum(axis=1)
dfFinalScore=dfFinalScore.sort_values(by='TotalScore',ascending=False)

# dfFinalScore['pctChangeScore']=dfFinalScore['TotalScore'].pct_change()*100
# dfFinalScore['pctChangeScore']=dfFinalScore['pctChangeScore'].round(0)
# dfFinalScore.fillna(value=0,inplace=True)

# dfFinalScore=dfFinalScore[['TotalScore','pctChangeScore','value_return_rank','value_std_rank','return_rank','std_rank']]
dfFinalScore=dfFinalScore[['TotalScore','value_return_rank','value_std_rank','return_rank','std_rank']]
dfFinalScore

,TotalScore,value_return_rank,value_std_rank,return_rank,std_rank
symbol,,,,,
^STOXX50E,22.0,5.0,6.0,5.0,6.0
ACWI,16.0,3.0,5.0,3.0,5.0
SPY,16.0,4.0,4.0,4.0,4.0
^N225,16.0,6.0,2.0,6.0,2.0
^BSESN,8.0,1.0,3.0,1.0,3.0
VNM,6.0,2.0,1.0,2.0,1.0


In [51]:
if isPriceExcel:
    print("Export outcoome as Excel")
    with pd.ExcelWriter(f"{report_prefix_name}_{timeframe}_{from_str}To{to_str}.xlsx") as writer:
        df_Pct_Price.to_excel(writer, sheet_name=f"{timeframe}")

        dfSumOfRankSummaryY24.to_excel(writer, sheet_name="FinalScoreOnlyRankY24")
        
        dfPct.to_excel(writer, sheet_name=f"{timeframe}-Return")
        dfRankStd.to_excel(writer, sheet_name=f"{timeframe}-RankStd")
        
        dfStd.to_excel(writer, sheet_name=f"{timeframe}-Std")
        dfRank.to_excel(writer, sheet_name=f"{timeframe}-RankReturn")
        
        
        dfValueSummary.to_excel(writer, sheet_name="Value-Summary")
        dfRankSummary.to_excel(writer, sheet_name="Rank-Summary")

        dfFinalScore.to_excel(writer, sheet_name="FinalScore")

Export outcoome as Excel
